In [1]:
import os
import urllib.request
import re
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
if not os.path.exists("the-verdict.txt"):
    url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt"
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [3]:
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

In [4]:
raw_text

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)\n\n"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it\'s going to send the value of my picture \'way up; but I don\'t think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing\'s lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn\'s "Moon-dancers" to say, with tears in her eyes: "We shall not look upon its like again"?\n\nWell!--even 

In [5]:
len(raw_text)

20479

In [6]:
text = "Hello, my name is Hrishi. I like coding."
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [word for word in preprocessed if word.strip()]
len(preprocessed)

4690

In [7]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [8]:
vocab = {token:integer for integer,token in enumerate(all_words)}
#vocab

In [9]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [word for word in preprocessed if word.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [10]:
tokenizer = SimpleTokenizerV1(vocab)

In [11]:
text = """ It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""

In [12]:
ids = tokenizer.encode(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [13]:
tokenizer.decode(tokenizer.encode(text))

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [14]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
len(vocab)

1132

In [15]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [word for word in preprocessed if word.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [16]:
tokenizer = SimpleTokenizerV2(vocab)
text = "Hello, my name is Hrishi sdhireby7n8qvf xkjovm90r4<|endoftext|>"
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, my <|unk|> is <|unk|> <|unk|> <|unk|>'

In [17]:
tokenizer = tiktoken.get_encoding("gpt2")
tokenizer.decode(tokenizer.encode(text, allowed_special= {"<|endoftext|>"}))

'Hello, my name is Hrishi sdhireby7n8qvf xkjovm90r4<|endoftext|>'

In [18]:
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
len(enc_text)

5145

In [19]:
enc_sample = enc_text[50:]
len(enc_sample)

5095

In [20]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [21]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "----->", tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


In [22]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special = {"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i+1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [23]:
def create_dataloader_v1(txt, batch_size = 4, max_length = 256, stride = 128, shuffle = True, drop_last = True, num_workers = 0):
    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )

    return dataloader

In [24]:
dataloader = create_dataloader_v1(raw_text, batch_size = 1, max_length = 4, stride = 4, shuffle = False)

dataiter = iter(dataloader)
first_batch = next(dataiter)
first_batch

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]

In [25]:
second_batch = next(dataiter)
second_batch

[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]

In [26]:
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length = 4, stride = 4, shuffle = False)

dataiter = iter(dataloader)
inputs, targets = next(dataiter)
print(f"Inputs: {inputs}")
print(f"Targets: {targets}")

Inputs: tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets: tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [27]:
input_ids = torch.tensor([2, 3, 5, 1])

In [28]:
output_dim = 256

embedding_layer = torch.nn.Embedding(tokenizer.n_vocab, output_dim)

In [29]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.4507,  0.6955, -1.0044,  ...,  1.7467,  0.8555,  0.9386],
        [ 0.9077,  1.4516,  0.7631,  ..., -0.4274, -0.4408,  0.1168],
        [ 0.1549,  0.6596, -0.1134,  ...,  1.3601, -0.3987, -0.6763],
        ...,
        [ 3.3629,  0.2980,  0.5351,  ...,  1.1264, -1.3251,  1.3003],
        [-1.3504,  0.8137,  0.2014,  ..., -0.0527, -1.8449,  0.6766],
        [-2.1629, -0.8592,  0.2810,  ..., -0.7243,  0.1610,  2.1090]],
       requires_grad=True)


In [30]:
embedding_layer(input_ids)

tensor([[ 0.1549,  0.6596, -0.1134,  ...,  1.3601, -0.3987, -0.6763],
        [-0.5198, -0.9044, -0.0791,  ..., -1.3611,  0.3250, -2.3784],
        [-0.5811, -1.1573, -1.0190,  ...,  1.1609,  1.1359, -0.4656],
        [ 0.9077,  1.4516,  0.7631,  ..., -0.4274, -0.4408,  0.1168]],
       grad_fn=<EmbeddingBackward0>)

In [31]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length = max_length, stride = max_length, shuffle = False)

dataiter = iter(dataloader)
inputs, targets = next(dataiter)

In [32]:
token_embeddings = embedding_layer(inputs)

In [33]:
token_embeddings.shape

torch.Size([8, 4, 256])

In [34]:
token_embeddings[0, 0]

tensor([ 1.6618e+00, -5.3588e-01,  9.6658e-02, -1.4797e+00,  4.4904e-01,
         1.4636e+00,  7.3901e-01, -1.0326e+00,  1.1452e+00, -1.6481e+00,
         4.6238e-01, -1.2260e+00, -1.3841e+00,  8.7766e-01, -7.4903e-01,
         1.1590e+00, -2.2549e-01,  4.3829e-01, -8.6002e-01,  8.0251e-01,
         1.5094e+00, -5.5633e-01,  1.3460e-01,  1.0513e+00,  1.0052e+00,
        -1.0061e+00, -1.0536e+00,  2.4632e-01,  1.1348e+00,  1.0769e+00,
        -7.9464e-01, -1.7009e+00,  3.1999e-01, -1.0349e+00,  1.5629e+00,
        -1.0033e+00, -6.5137e-01, -8.3610e-01,  2.1197e-01, -7.0868e-01,
         2.7596e-01, -3.8158e-01, -8.5280e-01, -1.4556e+00, -7.8473e-01,
        -1.7996e+00,  8.7553e-01, -8.1245e-01, -7.2008e-01,  3.8565e-01,
        -2.1837e+00, -3.1043e-01,  2.8949e-01,  4.3787e-01,  7.8359e-01,
         1.2147e+00,  4.8706e-01,  9.2464e-01, -1.2725e+00,  4.7903e-01,
         1.8346e+00,  4.8287e-01,  1.0208e+00, -1.0232e+00, -9.7494e-01,
        -1.2592e+00, -1.2096e+00,  6.2760e-01, -3.9

In [35]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [36]:
torch.arange(max_length)

tensor([0, 1, 2, 3])

In [37]:
pos_embedding_layer.weight

Parameter containing:
tensor([[-0.2211,  0.3026,  2.1867,  ..., -0.3307,  0.2512, -0.0236],
        [-0.4281, -1.4487, -0.0667,  ...,  0.4005, -1.6802,  0.2413],
        [-0.6053,  0.2560, -1.5217,  ...,  1.3820,  0.0470,  1.4569],
        [ 0.2306,  1.1210,  0.6219,  ...,  1.5985,  0.7454,  0.3675]],
       requires_grad=True)

In [38]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
pos_embeddings.shape

torch.Size([4, 256])

In [39]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
